In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import os

In [4]:
# change the directiory to root to allow importing of py files saved in algorithmictrading
os.chdir(r'..\\')
from algorithmictrading.secrets import IEX_CLOUD_API_TOKEN

# Data Loading - S&P 500 Index
The S&P 500 Index is one of the most common benchmarks for US Large Cap stocks. It tracks the performance of 500 of the largest companies in the United States.

You can substitute any list of tickers for this equal weight walk-through. The list of stocks should be aved in the *data* folder.

In [6]:
stocks = pd.read_csv(r'.\data\sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# Connecting to the IEX API
We will be using the free IEX Cloud API for the market data. Prices are purpsefully scrambled and are NOT meant for production!

[Documentation can be found here.](https://iexcloud.io/docs/api/#testing-sandbox)

We can use the base URL and pass `symbol` and `token` to connect to the API and pull in the correct data.

In [8]:
BASE_URL = 'https://sandbox.iexapis.com/stable'
symbol = 'AAPL'
stats = f'/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(BASE_URL+stats).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2173625669763,
 'week52high': 141.36,
 'week52low': 58.01,
 'week52change': 0.6933281163316503,
 'sharesOutstanding': 17462085410,
 'float': 0,
 'avg10Volume': 119688057,
 'avg30Volume': 117945720,
 'day200MovingAvg': 117.92,
 'day50MovingAvg': 129.95,
 'employees': 0,
 'ttmEPS': 3.29,
 'ttmDividendRate': 0.8400082494886704,
 'dividendYield': 0.006490665779938774,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-05',
 'nextEarningsDate': '0',
 'peRatio': 39.505951256412,
 'beta': 1.1720825462923594,
 'maxChangePercent': 48.9134602448249,
 'year5ChangePercent': 4.707256853794466,
 'year2ChangePercent': 2.506256873581805,
 'year1ChangePercent': 0.688802859169871,
 'ytdChangePercent': -0.030602956595569525,
 'month6ChangePercent': 0.3523184306009466,
 'month3ChangePercent': 0.0384143316205799,
 'month1ChangePercent': 0.05351798877141296,
 'day30ChangePercent': 0.05224376040739527,
 'day5ChangePercent': -0.01694180525930723}

In [9]:
# momentum stat
data['year1ChangePercent']

0.688802859169871

# Making Batch API Calls
As seen in the [equal weight SP 500 notebook](.\notebooks\sp500-equal-weight.ipynb), making a single http request is really slow. We are much better served breaking up our security list into small batches. The IEX API limits 100 symbols per batch, we we will make 6 http requests.

In [10]:
def make_chunks(df):
     return np.array_split(df['Ticker'].to_list(), np.ceil(len(df) / 100))

def get_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        one_year_changes = [data[k]['stats']['year1ChangePercent'] for k in data.keys()]
        df = pd.DataFrame({'ticker': tickers, 'latest_price': latestprices, '1_year_change': one_year_changes})
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)


In [11]:
df = get_data_batch(stocks)
df.head()

,ticker,latest_price,1_year_change
0,A,129.830,0.468475
1,AAL,15.865,-0.450572
2,AAP,174.940,0.200965
3,AAPL,129.750,0.682180
4,ABBV,113.080,0.303073


In [22]:
def transform_momentum_df(df):
    df = df.copy()
    return (df.sort_values('1_year_change', ascending=False)
              .reset_index(drop=True)
              .iloc[:50]
           )

In [25]:
mom_df = transform_momentum_df(df)
mom_df.head()

,ticker,latest_price,1_year_change
0,CARR,42.290,2.585607
1,ALB,182.680,1.620473
2,LB,47.567,1.468541
3,FCX,31.570,1.429277
4,NVDA,547.490,1.237922
